# OBIA Feature Extraction with Sentinel-2 Imagery


This notebook performs object-based feature extraction from Sentinel-2 imagery using the `nickyspatial` library.

Steps included:
1. Load Sentinel-2 bands (Red and NIR)
2. Compute NDVI
3. Segment the image using SLIC
4. Compute zonal statistics (mean NDVI)
5. Extract geometry features (area, perimeter, compactness)
6. Compute GLCM texture features (contrast)
7. Save the results as GeoJSON and visualize
    

In [ ]:

import rasterio
import numpy as np
import geopandas as gpd
from skimage.feature import greycomatrix, greycoprops
import matplotlib.pyplot as plt

from nickyspatial import SlicSegmentation, attach_basic_stats, attach_shape_metrics, layer_to_vector


## Load Sentinel-2 Red and NIR bands from multiband TIFF

In [ ]:

# Replace 'sentinel2.tif' with your actual multiband file
multiband_path = "sentinel2.tif"

with rasterio.open(multiband_path) as src:
    print(f"Number of bands: {src.count}")

    red = src.read(4).astype("float32")  # Band 4: Red
    nir = src.read(8).astype("float32")  # Band 8: NIR

    profile = src.profile
    transform = src.transform


## Compute NDVI

In [ ]:

ndvi = (nir - red) / (nir + red)
ndvi = np.clip(ndvi, -1, 1)

plt.imshow(ndvi, cmap='RdYlGn')
plt.title("NDVI")
plt.colorbar()
plt.show()


## Segment NDVI using SLIC from nickyspatial

In [ ]:

slic = SlicSegmentation(n_segments=1000, compactness=10)
segments = slic.fit(ndvi)

plt.imshow(segments, cmap='tab20')
plt.title("Segmented Image (SLIC)")
plt.colorbar()
plt.show()


## Extract Zonal Statistics and Geometry Features

In [ ]:

# Build layer object and convert to vector
layer = {'raster': ndvi, 'segments': segments, 'transform': transform}
gdf = layer_to_vector(layer)

# Add mean NDVI
gdf = attach_basic_stats(gdf, ndvi, 'mean_ndvi')

# Add geometry metrics
gdf = attach_shape_metrics(gdf)


## Compute Texture Features (GLCM Contrast from NDVI)

In [ ]:

ndvi_int = ((ndvi + 1) * 127.5).astype('uint8')  # Scale to 0–255
glcm = greycomatrix(ndvi_int, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
contrast = greycoprops(glcm, 'contrast')[0, 0]

# Assign global contrast to all objects (optional: do per-object if needed)
gdf['glcm_contrast'] = contrast


## Save Results as GeoJSON

In [ ]:

gdf.to_file("obia_features.geojson", driver="GeoJSON")


## Visualize Mean NDVI per Object

In [ ]:

gdf.plot(column='mean_ndvi', cmap='YlGn', legend=True)
plt.title("Mean NDVI per Object")
plt.show()
